<a href="https://colab.research.google.com/gist/zemogle/fa53f58455ac3cb00ea836c7f46915c4/colour-imaging-with-las-cumbres-observatory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



## Code Imports
You don't need to expand this section. You can just run the whole thing and it will import all the functions you need.



In [2]:
!pip install fits_align
!pip install fits2image
from astropy.io import fits
import numpy as np  
from fits_align.ident import make_transforms
from fits_align.align import affineremap
from ipywidgets import widgets
import requests
from ipywidgets import interactive, interact, fixed
import ipywidgets as widgets
from PIL import Image as pimage
from io import BytesIO
from fits2image.scaling import stack_images, percentile_scale, recalculate_median
from IPython.display import Image

ARCHIVE_API = 'https://archive-api.lco.global/'
THUMBNAIL_SIZE = (1000,1000)

  Created wheel for fits-align: filename=fits_align-0.4.3-py3-none-any.whl size=29456 sha256=89d18366b143853d462099047879048daf9dd1bd6981b904170ca1a45538e8ac
  Stored in directory: /root/.cache/pip/wheels/9c/6c/8b/a5d3d2253c370130d0a7ee1cfb3f66168f1c9ee618c0f10499
Successfully built fits-align


In [3]:
def rvb_frames(frames):

    FILTERS = {
        'red': ['R', 'rp','ip'],
        'visual': ['V','rp'],
        'blue': ['B','gp'],
    }

    selected_frames = []
    for color in ['red', 'visual', 'blue']:
        try:
            selected_frames.append(
                next(f for f in frames if f['FILTER'] in FILTERS[color])
            )
        except StopIteration:
            print('Filters for colour not found')
            return None
    return selected_frames

In [4]:
def get_fits_data(url):
    with fits.open(url) as hdul:
        for hdu in hdul:
            if len(np.shape(hdu)) == 2:
                return hdu.data

def display_img_array(imdata):
    bio = BytesIO()
    rgb_cube = np.dstack(imdata).astype(np.uint8)
    im = pimage.fromarray(rgb_cube)
    if im.mode != 'RGB':
        im = im.convert('RGB')
    im.thumbnail(THUMBNAIL_SIZE, pimage.ANTIALIAS)
    im.save(bio, 'png')
    display(Image(bio.getvalue(), format='png', retina=True))
    return

In [5]:
def frames_for_requestnum(reqnum):
    headers = {
        'Authorization': token.value
    }
    frames = requests.get(
        '{0}frames/?REQNUM={1}'.format(ARCHIVE_API, reqnum),
        headers=headers
    ).json()['results']
    if any(f for f in frames if f['RLEVEL'] == 91):
        rlevel = 91
    elif any(f for f in frames if f['RLEVEL'] == 11):
        rlevel = 11
    else:
        rlevel = 0
    return [f for f in frames if f['RLEVEL'] == rlevel]

In [6]:
def import_images(reqnum):
    print('Starting download...',end="\r")
    frames = frames_for_requestnum(reqnum)
    sframes = rvb_frames(frames)
    imdata = [] 
    
    for f in sframes:
        imdata.append(get_fits_data(f['url']))
    print('Images Download complete',end="\r")
    return imdata

In [7]:
def scale_data(percentile, imgarr, stretch=False):
    print('Scaling', end="\r")
    tmparr = []
    for img in imgarr:
        if stretch:
          img = np.arcsinh(img)
        tmparr.append(recalculate_median(img, percentile=percentile))
    display_img_array(tmparr)
    return percentile

# Colour Imager

Enter your API Key (a long string of numbers and letters) from the LCO [Observing Portal > Profile](https://observe.lco.global/accounts/profile/)

In [8]:
token = widgets.Text()
token

Text(value='')

The next cell collects the data files and makes sure you have 3 images which can be used to make a colour image

Enter the **sub-request number** of your request (not the request group number)

In [ ]:
imdata = interactive(import_images, {'manual': True}, reqnum=widgets.IntText())
display(imdata)


In [ ]:
percentile = widgets.Dropdown(
    options=[85, 98, 99, 99.5, 99.95],
    value=99.5,
    description='Percentile:',
    disabled=False,
)
stretch = widgets.ToggleButton(value=True, description='Stretch', tooltip='Stretch the range of brightness',icon='expand')
ui = widgets.HBox([percentile, stretch])
out = widgets.interactive_output(scale_data, {'percentile': percentile, 'imgarr': fixed(imdata.result), 'stretch': stretch})
display(ui, out)